In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import *
from sklearn.metrics import *
from sklearn.cluster import KMeans
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.multitest import multipletests
import itertools

In [ ]:
normalized_lipids = pd.read_csv('normalized_lipids.csv')
normalized_rnaseq = pd.read_csv('normalized_rnaseq.csv')
metadata = pd.read_csv('t2d_metadata.csv')

In [ ]:
metadata.head(2)

In [ ]:
normalized_rnaseq

In [ ]:
normalized_lipids

In [ ]:
def data_cleaning(dataframe):
    col = pd.DataFrame({'col':dataframe.isnull().sum().keys()})
    missing_val = pd.DataFrame({'miss_val':dataframe.isnull().sum().values})
    miss = pd.concat([col,missing_val],axis = 1)
    null_df = miss[miss['miss_val'] != 0]
    if null_df.shape[0] == 0:
        print("------------------------------------------------")
        print("This Dataframe does not contains any null values")
        print("------------------------------------------------")
    else:
        print("------------------------------------------------")
        print("This Dataframe have some missing values...")
        print("------------------------------------------------")
        print("------------------------------------------------")
        print("Checking for the columns which contains missing values")
        print("------------------------------------------------")
        print('Out of total' , dataframe.shape[0],'samples we have the following missing values :')
        col_mis = list(null_df['col'])
        for val in col_mis:
            val_count = int(null_df[null_df['col'] == val]['miss_val'].iloc[0])
            percent = val_count / dataframe.shape[0] * 100
            print(f"{percent:.2f}% values are missing in: {val}")
            
        print("------------------------------------------------") 
        
        
        
    
        
    return miss

In [ ]:
data_cleaning(metadata)

In [ ]:
def impute_biological_data(df):
    """
    Comprehensive imputation strategy for biological data with detailed methods for each column
    """
    imputed_df = df.copy()
    
    # 1. Age (1.48% missing)
    imputed_df['Age'] = imputed_df['Age'].fillna(imputed_df['Age'].median())
    
    # 2. BMI (1.48% missing)
    imputed_df['BMI'] = imputed_df['BMI'].fillna(imputed_df['BMI'].median())
    
    # 3. HbA1c (2.96% missing)
    knn_imputer = KNNImputer(n_neighbors=5, weights='distance')
    imputed_df['HbA1c'] = knn_imputer.fit_transform(imputed_df[['HbA1c', 'BMI', 'Age']])[:,0]
    
    # 4. OGTT (43.70% missing)
    # Modified MICE imputer without sample_posterior
    mice_imputer = IterativeImputer(
        random_state=42,
        max_iter=10,
        min_value=0,
        estimator=RandomForestRegressor(
            n_estimators=100,
            random_state=42,
            n_jobs=-1  # Use all available cores
        )
    )
    
    # Include relevant predictors for OGTT
    predictors = ['OGTT', 'HbA1c', 'BMI', 'Age']
    temp_data = imputed_df[predictors].copy()
    imputed_values = mice_imputer.fit_transform(temp_data)
    imputed_df['OGTT'] = imputed_values[:,0]
    
    # Validate biological ranges
    imputed_df = validate_biological_ranges(imputed_df)
    
    return imputed_df

def validate_biological_ranges(df):
    """
    Ensure imputed values are within biological ranges
    """
    # Create a copy to avoid SettingWithCopyWarning
    df = df.copy()
    
    # Age constraints
    df.loc[df['Age'] < 0, 'Age'] = df['Age'].median()
    df.loc[df['Age'] > 120, 'Age'] = df['Age'].median()
    
    # BMI constraints
    df.loc[df['BMI'] < 10, 'BMI'] = df['BMI'].median()
    df.loc[df['BMI'] > 100, 'BMI'] = df['BMI'].median()
    
    # HbA1c constraints (typical range 3-15%)
    df.loc[df['HbA1c'] < 3, 'HbA1c'] = 3
    df.loc[df['HbA1c'] > 15, 'HbA1c'] = 15
    
    # OGTT constraints (typical range 0-500 mg/dL)
    df.loc[df['OGTT'] < 0, 'OGTT'] = 0
    df.loc[df['OGTT'] > 500, 'OGTT'] = 500
    
    return df

def analyze_imputation_results(original_df, imputed_df):
    """
    Comprehensive analysis of imputation results
    """
    columns = ['Age', 'BMI', 'HbA1c', 'OGTT']
    
    for col in columns:
        print(f"\nAnalysis for {col}:")
        print("-" * 50)
        
        # Missing value analysis
        missing_count = original_df[col].isnull().sum()
        missing_percent = (missing_count / len(original_df)) * 100
        print(f"Original missing values: {missing_count} ({missing_percent:.2f}%)")
        
        # Basic statistics
        orig_stats = original_df[col].describe()
        imp_stats = imputed_df[col].describe()
        
        stats_comparison = pd.DataFrame({
            'Original': orig_stats,
            'Imputed': imp_stats
        })
        print("\nDescriptive Statistics Comparison:")
        print(stats_comparison)
        
        # Distribution plot
        plt.figure(figsize=(10, 6))
        
        # Plot original data distribution
        sns.kdeplot(data=original_df[col].dropna(), 
                   label='Original', alpha=0.5)
        
        # Plot imputed data distribution
        sns.kdeplot(data=imputed_df[col], 
                   label='Imputed', alpha=0.5)
        
        plt.title(f'Distribution Comparison for {col}')
        plt.legend()
        plt.show()

if __name__ == "__main__":
    try:
        imputed_df = impute_biological_data(metadata)
        
        
        # Analyze results
        analyze_imputation_results(metadata, imputed_df)
        
        print("\nImputation completed successfully!")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")


In [ ]:
from scipy.stats import probplot

In [ ]:


def impute_ogtt_preserving_distribution(df, column='OGTT'):
    """
    Impute OGTT values while preserving the original distribution
    """
    df_copy = df.copy()
    
    # 1. Separate missing and non-missing data
    missing_mask = df_copy[column].isnull()
    known_values = df_copy[~missing_mask][column]
    
    # 2. Prepare predictors for imputation
    predictors = ['HbA1c', 'BMI', 'Age']
    
    # 3. Standardize predictors
    scaler = StandardScaler()
    X_known = scaler.fit_transform(df_copy[~missing_mask][predictors])
    X_missing = scaler.transform(df_copy[missing_mask][predictors])
    
    # 4. Train Random Forest model on non-missing data
    rf_model = RandomForestRegressor(
        n_estimators=100,
        random_state=42,
        max_features='sqrt',
        min_samples_leaf=3
    )
    rf_model.fit(X_known, known_values)
    
    # 5. Generate multiple predictions for each missing value
    n_iterations = 10
    all_predictions = []
    
    for i in range(n_iterations):
        predictions = rf_model.predict(X_missing)
        all_predictions.append(predictions)
    
    # 6. Add random noise based on original distribution
    std_dev = known_values.std() * 0.1  # 10% of original std
    final_predictions = np.mean(all_predictions, axis=0)
    noise = np.random.normal(0, std_dev, size=len(final_predictions))
    final_predictions += noise
    
    # 7. Ensure predictions follow original distribution
    def match_distribution(original_values, predicted_values):
        """
        Adjust predicted values to better match the original distribution
        """
        # Get original percentiles
        orig_percentiles = np.percentile(original_values, np.linspace(0, 100, 20))
        pred_percentiles = np.percentile(predicted_values, np.linspace(0, 100, 20))
        
        # Create interpolation function
        from scipy.interpolate import interp1d
        transform_func = interp1d(pred_percentiles, orig_percentiles, 
                                bounds_error=False, fill_value='extrapolate')
        
        # Transform predicted values
        adjusted_predictions = transform_func(predicted_values)
        
        # Ensure values are within biological limits
        adjusted_predictions = np.clip(adjusted_predictions, 0, 500)
        
        return adjusted_predictions
    
    # Apply distribution matching
    final_predictions = match_distribution(known_values, final_predictions)
    
    # 8. Insert predictions back into dataframe
    df_copy.loc[missing_mask, column] = final_predictions
    
    return df_copy

def impute_biological_data(df):
    """
    Comprehensive imputation strategy for biological data
    """
    imputed_df = df.copy()
    
    # 1. Age (1.48% missing)
    imputed_df['Age'] = imputed_df['Age'].fillna(imputed_df['Age'].median())
    
    # 2. BMI (1.48% missing)
    imputed_df['BMI'] = imputed_df['BMI'].fillna(imputed_df['BMI'].median())
    
    # 3. HbA1c (2.96% missing)
    knn_imputer = KNNImputer(n_neighbors=5, weights='distance')
    imputed_df['HbA1c'] = knn_imputer.fit_transform(imputed_df[['HbA1c', 'BMI', 'Age']])[:,0]
    
    # 4. OGTT (43.70% missing) - Using the new distribution-preserving method
    imputed_df = impute_ogtt_preserving_distribution(imputed_df)
    
    return imputed_df

def analyze_distributions(original_df, imputed_df, column):
    """
    Analyze and visualize the distributions before and after imputation
    """
  
    
    plt.figure(figsize=(15, 5))
    
    # Plot 1: Density plots
    plt.subplot(131)
    sns.kdeplot(data=original_df[column].dropna(), label='Original', alpha=0.5)
    sns.kdeplot(data=imputed_df[column], label='Imputed', alpha=0.5)
    plt.title(f'{column} Distribution Comparison')
    plt.legend()
    
    # Plot 2: Q-Q plot
    plt.subplot(132)
    probplot(imputed_df[column], dist="norm", plot=plt)
    plt.title('Q-Q Plot of Imputed Data')
    
    # Plot 3: Box plots
    plt.subplot(133)
    plt.boxplot([original_df[column].dropna(), imputed_df[column]], 
                labels=['Original', 'Imputed'])
    plt.title('Box Plot Comparison')
    
    plt.tight_layout()
    plt.show()
    
    # Statistical tests
    print(f"\nStatistical Analysis for {column}:")
    print("-" * 50)
    
    # Kolmogorov-Smirnov test
    ks_stat, p_value = stats.ks_2samp(
        original_df[column].dropna(),
        imputed_df[column]
    )
    print(f"KS test p-value: {p_value:.4f}")
    
    # Basic statistics comparison
    stats_comparison = pd.DataFrame({
        'Original': original_df[column].describe(),
        'Imputed': imputed_df[column].describe()
    })
    print("\nDescriptive Statistics:")
    print(stats_comparison)

# Usage example:
if __name__ == "__main__":
    # Impute the data
    imputed_df = impute_biological_data(metadata)
    
    # Analyze OGTT distribution
    analyze_distributions(metadata, imputed_df, 'OGTT')
    


In [ ]:
imputed_metadata = imputed_df

In [ ]:

def visualize_outliers(df):
    """
    Plots distributions and boxplots to visualize outliers in numeric columns of a DataFrame.

    Parameters:
    - df (pd.DataFrame): Input DataFrame
    """

    numeric_cols = df.select_dtypes(include='number').columns

    for col in numeric_cols:
        plt.figure(figsize=(12, 5))

        # Distribution plot
        plt.subplot(1, 2, 1)
        sns.histplot(df[col], kde=True, bins=30, color='skyblue', alpha=0.6)
        plt.title(f'Distribution: {col}')

        # Boxplot
        plt.subplot(1, 2, 2)
        sns.boxplot(x=df[col], color='salmon')
        plt.title(f'Boxplot: {col}')

        plt.tight_layout()
        plt.show()


In [ ]:
visualize_outliers(normalized_lipids)

In [ ]:
visualize_outliers(normalized_rnaseq)

In [ ]:
import pandas as pd
import numpy as np

def remove_outliers_iqr(df, fill_method=None):
    """
    Removes outlier expression values per gene using IQR method.
    
    Args:
        df (pd.DataFrame): Normalized RNA-seq data (genes as rows, samples as columns).
        fill_method (str or None): 'median', 'mean', or None (keep NaN).

    Returns:
        pd.DataFrame: DataFrame with outlier values removed or imputed.
    """
    cleaned_df = df.copy()
    col_1 = cleaned_df.iloc[0:,0]
    cleaned_df = cleaned_df.iloc[0:,1:]

    for gene in cleaned_df.index:
        row = cleaned_df.loc[gene]
        Q1 = row.quantile(0.25)
        Q3 = row.quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        # Find outliers
        mask = (row < lower) | (row > upper)

        # Replace outliers
        if fill_method == 'median':
            cleaned_df.loc[gene, mask] = row.median()
        elif fill_method == 'mean':
            cleaned_df.loc[gene, mask] = row.mean()
        else:
            cleaned_df.loc[gene, mask] = np.nan
    cleaned_df = pd.concat([col_1,cleaned_df],axis = 1)

    return cleaned_df


In [ ]:
normalized_lipid_outlier_removed = remove_outliers_iqr(normalized_lipids,fill_method='median')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns

def detect_and_remove_outlier_samples(df, method="PCA", n_clusters=3, threshold_percentile=95):
    """
    Detect and remove outlier samples using PCA or KMeans clustering.
    
    Args:
        df (pd.DataFrame): Normalized RNA-seq data (genes as rows, samples as columns).
        method (str): "PCA" or "Clustering" method for outlier detection.
        n_clusters (int): Number of clusters for KMeans (if method is "Clustering").
        threshold_percentile (int): Percentile threshold to classify outliers.
        
    Returns:
        cleaned_df (pd.DataFrame): DataFrame with outlier samples removed.
        outlier_samples (list): List of column names of outlier samples.
        explained_variance (numpy.array): Explained variance ratios from PCA
    """
    try:
        # Make a copy of the original dataframe
        df_copy = df.copy()
        
        # Store the first column (usually gene names/IDs)
        first_col_name = df_copy.columns[0]
        first_col = df_copy[[first_col_name]]
        
        # Get the data columns (excluding the first column)
        data_cols = df_copy.columns[1:]
        data_df = df_copy[data_cols]
        
        # Transpose data for scaling (samples as rows)
        df_transposed = data_df.T
        
        # Standardize the data
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(df_transposed)
        
        outlier_samples = []
        explained_variance = None
        
        if method == "PCA":
            # Apply PCA
            pca = PCA(n_components=2)
            pca_result = pca.fit_transform(scaled_data)
            explained_variance = pca.explained_variance_ratio_
            
            # Calculate distances from origin
            distances = np.sqrt(np.sum(pca_result**2, axis=1))
            
            # Define threshold
            threshold = np.percentile(distances, threshold_percentile)
            
            # Identify outliers
            outlier_mask = distances > threshold
            outlier_samples = data_df.columns[outlier_mask].tolist()
            
            # Plotting
            plt.figure(figsize=(10, 8))
            plt.scatter(pca_result[~outlier_mask, 0], pca_result[~outlier_mask, 1], 
                       c='blue', label='Normal Samples')
            plt.scatter(pca_result[outlier_mask, 0], pca_result[outlier_mask, 1], 
                       c='red', label='Outliers')
            plt.title('PCA of RNA-seq Samples')
            plt.xlabel(f'PC1 ({explained_variance[0]:.2%} variance explained)')
            plt.ylabel(f'PC2 ({explained_variance[1]:.2%} variance explained)')
            plt.legend()
            plt.show()
            
        elif method == "Clustering":
            # Apply KMeans
            kmeans = KMeans(n_clusters=n_clusters, random_state=42)
            cluster_labels = kmeans.fit_predict(scaled_data)
            
            # Calculate distances to nearest cluster center
            distances = np.min(kmeans.transform(scaled_data), axis=1)
            
            # Define threshold
            threshold = np.percentile(distances, threshold_percentile)
            
            # Identify outliers
            outlier_mask = distances > threshold
            outlier_samples = data_df.columns[outlier_mask].tolist()
            
            # Apply PCA for visualization
            pca = PCA(n_components=2)
            pca_result = pca.fit_transform(scaled_data)
            explained_variance = pca.explained_variance_ratio_
            
            # Plotting
            plt.figure(figsize=(10, 8))
            scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], 
                                c=cluster_labels, cmap='Set2')
            plt.scatter(pca_result[outlier_mask, 0], pca_result[outlier_mask, 1], 
                       c='red', marker='x', s=100, label='Outliers')
            plt.title('Clustering of RNA-seq Samples')
            plt.xlabel(f'PC1 ({explained_variance[0]:.2%} variance explained)')
            plt.ylabel(f'PC2 ({explained_variance[1]:.2%} variance explained)')
            plt.legend()
            plt.colorbar(scatter, label='Cluster')
            plt.show()
        
        # Print outlier information
        print(f"\nNumber of outliers detected: {len(outlier_samples)}")
        print("Outlier samples:", outlier_samples)
        
        # Remove outliers
        columns_to_keep = [col for col in data_df.columns if col not in outlier_samples]
        cleaned_data = data_df[columns_to_keep]
        
        # Combine first column with cleaned data
        cleaned_df = pd.concat([first_col, cleaned_data], axis=1)
        
        # Print shape information
        print(f"\nOriginal shape: {df.shape}")
        print(f"Cleaned shape: {cleaned_df.shape}")
        
        return cleaned_df, outlier_samples, explained_variance
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None

# Example usage with debugging information
def test_outlier_removal(df, method="PCA", threshold_percentile=95):
    """
    Test the outlier removal function with detailed debugging information
    """
    print("Starting outlier detection...")
    print(f"Input DataFrame shape: {df.shape}")
    print(f"Input DataFrame columns: {df.columns.tolist()[:5]}... (first 5)")
    
    cleaned_df, outliers, exp_var = detect_and_remove_outlier_samples(
        df,
        method=method,
        threshold_percentile=threshold_percentile
    )
    
    if cleaned_df is not None:
        print("\nOutlier removal completed successfully")
        print(f"Number of columns removed: {len(outliers)}")
        print(f"Output DataFrame shape: {cleaned_df.shape}")
        print(f"Output DataFrame columns: {cleaned_df.columns.tolist()[:5]}... (first 5)")
        
        # Verify removal
        if len(outliers) > 0:
            for outlier in outliers:
                if outlier in cleaned_df.columns:
                    print(f"WARNING: Outlier {outlier} still present in cleaned data!")
                else:
                    print(f"Confirmed: Outlier {outlier} successfully removed")
    
    return cleaned_df, outliers, exp_var





In [ ]:
normalized_rnaseq_outlier_removed, outliers,explained_variance = test_outlier_removal(
    normalized_rnaseq,
    method="PCA",
    threshold_percentile=95
)


In [ ]:
normalized_rnaseq_outlier_removed

In [ ]:
visualize_outliers(normalized_lipid_outlier_removed)

In [ ]:
visualize_outliers(normalized_rnaseq_outlier_removed)

In [ ]:
def data_merging(rnaseq,lipid,metadat):
    col_lipid = list(lipid.T.reset_index().iloc[0,0:])
    lipid = lipid.T.iloc[1:,]
    lipid.columns = col_lipid[1:]#.reset_index()#.rename(columns = {'index':'genes'})
    lipid = lipid.reset_index().rename(columns = {'index':'#NAME'})
    lipid_merge = lipid.merge(metadat,on = '#NAME',how = 'inner')
    
    rnaseq = rnaseq.T
    col_rnaseq = list(rnaseq.iloc[0,0:])
   
    rnaseq.columns = col_rnaseq#.reset_index()#.rename(columns = {'index':'genes'})
    rnaseq = rnaseq.iloc[1:].reset_index().rename(columns = {'index':'#NAME'})
    rnaseq_merge = rnaseq.merge(metadat,on = '#NAME',how = 'inner')
   
    return rnaseq_merge,lipid_merge

In [ ]:
rnaseq_merge,lipid_merge = data_merging(normalized_rnaseq,normalized_lipids,imputed_metadata)

In [ ]:
lipid_merge.to_csv("lipid_merge.csv", index=False)


In [ ]:


def assess_samplewise_distribution_and_normality(df, num_samples=5):
    """
    Assess distribution, skewness, and normality of RNA-seq expression across samples (columns).

    Args:
        df (pd.DataFrame): Normalized RNA-seq dataframe (samples as columns).
        num_samples (int): Number of random samples (columns) to visualize.

    Returns:
        normality_df (pd.DataFrame): Summary table with skewness and normality p-values.
    """
    # Ensure index is not 'Unnamed' column
    if df.columns[0] == "Unnamed: 0":
        df = df.drop(columns=["Unnamed: 0"])

    results = []
    sampled_cols = np.random.choice(df.columns, size=min(num_samples, df.shape[1]), replace=False)

    for sample in sampled_cols:
        data = df[sample].dropna()
        gene_skew = skew(data)
        stat, p_value = shapiro(data)

        results.append({
            'Sample': sample,
            'Skewness': gene_skew,
            'Shapiro_pval': p_value,
            'Is_Normal (p>0.05)': p_value > 0.05
        })

        # Plot distribution
        plt.figure(figsize=(12, 5))

        # Histogram + KDE
        plt.subplot(1, 2, 1)
        sns.histplot(data, kde=True, bins=30, color='skyblue', edgecolor='black')
        plt.axvline(data.mean(), color='red', linestyle='--', label='Mean')
        plt.title(f'Distribution for Sample: {sample}')
        plt.legend()

        # QQ Plot
        plt.subplot(1, 2, 2)
        probplot(data, dist="norm", plot=plt)
        plt.title(f'QQ Plot for Sample: {sample}')

        plt.tight_layout()
        plt.show()

    normality_df = pd.DataFrame(results)
    return normality_df


In [ ]:
normality_df_rnaseq = assess_samplewise_distribution_and_normality(normalized_rnaseq_outlier_removed, num_samples=len(normalized_rnaseq_outlier_removed))

In [ ]:
normality_df_lipid = assess_samplewise_distribution_and_normality(normalized_lipid_outlier_removed, num_samples=len(normalized_lipid_outlier_removed))

In [ ]:
normality_df_lipid

In [ ]:
normality_df_rnaseq

In [ ]:

def apply_transformations_and_check_normality(df, num_samples=5):
    """
    Apply various transformations sample-wise and assess normality.

    Args:
        df (pd.DataFrame): Normalized RNA-seq data (samples as columns).
        num_samples (int): Number of random samples to visualize.

    Returns:
        results_df (pd.DataFrame): Summary of skewness and p-values post-transformation.
    """

    # Drop identifier column if present
    if df.columns[0] == "Unnamed: 0":
        df = df.drop(columns=["Unnamed: 0"])

    transformations = {
        'Log1p': lambda x: np.log1p(x),
        'Sqrt': lambda x: np.sqrt(np.abs(x)),
        'Reciprocal': lambda x: 1 / (x + 1e-6),  # Add small constant to avoid /0
        'Z-Score': lambda x: zscore(x, nan_policy='omit')
    }

    sample_cols = np.random.choice(df.columns, size=min(num_samples, len(df.columns)), replace=False)

    all_results = []

    for sample in sample_cols:
        data = df[sample].dropna()

        for name, func in transformations.items():
            try:
                transformed = func(data)

                # Stats
                sk = skew(transformed)
                pval = shapiro(transformed)[1]

                all_results.append({
                    "Sample": sample,
                    "Transformation": name,
                    "Skewness": sk,
                    "Shapiro_pval": pval,
                    "Is_Normal (p>0.05)": pval > 0.05
                })

                # Plotting
                plt.figure(figsize=(12, 4))

                # Histogram + KDE
                plt.subplot(1, 2, 1)
                sns.histplot(transformed, kde=True, bins=30, color='skyblue', edgecolor='black')
                plt.axvline(transformed.mean(), color='red', linestyle='--', label='Mean')
                plt.title(f'{name} - Distribution of {sample}')
                plt.legend()

                # QQ Plot
                plt.subplot(1, 2, 2)
                probplot(transformed, dist="norm", plot=plt)
                plt.title(f'{name} - QQ Plot of {sample}')

                plt.tight_layout()
                plt.show()

            except Exception as e:
                print(f"Transformation {name} failed on {sample}: {e}")

    results_df = pd.DataFrame(all_results)
    return results_df


In [ ]:
transformation_rnaseq = apply_transformations_and_check_normality(normalized_rnaseq_outlier_removed,len(normalized_rnaseq_outlier_removed))

In [ ]:
transformation_lipid = apply_transformations_and_check_normality(normalized_lipid_outlier_removed,len(normalized_lipid_outlier_removed))

In [ ]:
import pandas as pd

# Summarize results for a transformation dataset
def summarize_transformations(transformation_df, dataset_name="Dataset"):
    summary = (
        transformation_df
        .groupby("Transformation")
        .agg(
            Avg_Skewness=('Skewness', 'mean'),
            Std_Skewness=('Skewness', 'std'),
            Normal_Count=('Is_Normal (p>0.05)', 'sum'),
            Total_Samples=('Is_Normal (p>0.05)', 'count')
        )
        .reset_index()
    )
    summary["% Normal (p > 0.05)"] = (summary["Normal_Count"] / summary["Total_Samples"]) * 100
    summary = summary.sort_values(["% Normal (p > 0.05)", "Avg_Skewness"], ascending=[False, True])

    print(f"\n===== Summary for {dataset_name} =====")
    print(summary)
    return summary

# Apply it to both datasets (assuming they're defined)
summary_rnaseq = summarize_transformations(transformation_rnaseq, "RNA-seq")
summary_lipid = summarize_transformations(transformation_lipid, "Lipidomics")


In [ ]:
rnaseq_merge

In [ ]:
from scipy.stats import kruskal

In [ ]:
from scipy.stats import mannwhitneyu, kruskal


In [ ]:
from scipy.stats import kruskal, mannwhitneyu
from statsmodels.stats.multitest import multipletests
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
def analyze_expression_by_diabetes_group(df, group_col='Diagnosis'):
    df = df.copy()

    # Separate metadata and features
    metadata = df[[group_col]]
    expression = df.drop(columns=['#NAME', group_col, 'Age', 'OGTT', 'HbA1c', 'BMI'])

    results = []

    for feature in expression.columns:
        temp_df = pd.concat([metadata, expression[[feature]]], axis=1)
        groups = [group[feature].dropna().values for name, group in temp_df.groupby(group_col)]

        # Kruskal-Wallis Test
        stat, p = kruskal(*groups)
        results.append({'Feature': feature, 'Kruskal_Wallis_stat': stat, 'Kruskal_Wallis_pval': p})

    kruskal_df = pd.DataFrame(results)
    kruskal_df['adj_pval'] = multipletests(kruskal_df['Kruskal_Wallis_pval'], method='fdr_bh')[1]

    # Visualize top 3 significant features
    top_features = kruskal_df.sort_values('adj_pval').head(3)['Feature']

    for feat in top_features:
        plt.figure(figsize=(10, 5))
        sns.boxplot(data=df, x=group_col, y=feat, palette='Set3')
        plt.title(f'Expression Distribution of {feat} by {group_col}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    # Pairwise Mann-Whitney U test
    pairwise_results = []
    for feature in top_features:
        temp_df = pd.concat([metadata, expression[[feature]]], axis=1)
        for (g1, g2) in itertools.combinations(temp_df[group_col].unique(), 2):
            group1_vals = temp_df[temp_df[group_col] == g1][feature].dropna()
            group2_vals = temp_df[temp_df[group_col] == g2][feature].dropna()
            stat, pval = mannwhitneyu(group1_vals, group2_vals, alternative='two-sided')
            pairwise_results.append({
                'Feature': feature,
                'Group1': g1,
                'Group2': g2,
                'U_stat': stat,
                'p_value': pval
            })

    pairwise_df = pd.DataFrame(pairwise_results)
    pairwise_df['adj_pval'] = multipletests(pairwise_df['p_value'], method='fdr_bh')[1]

    return kruskal_df.sort_values('adj_pval'), pairwise_df.sort_values('adj_pval')

In [ ]:
kruskal_results_rnaseq, pairwise_results_rnaseq = analyze_expression_by_diabetes_group(rnaseq_merge)

In [ ]:
print(kruskal_results_rnaseq.head())
print(pairwise_results_rnaseq.head())

In [ ]:
from scipy.stats import kruskal, mannwhitneyu
from statsmodels.stats.multitest import multipletests
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def analyze_lipid_by_diabetes_group(df, group_col='Diagnosis'):
    df = df.copy()

    # Separate metadata and lipid features
    metadata = df[[group_col]]
    lipid = df.drop(columns=['#NAME', group_col, 'Age', 'OGTT', 'HbA1c', 'BMI'])

    # Ensure all lipid columns are numeric
    lipid = lipid.apply(pd.to_numeric, errors='coerce')

    results = []

    for feature in lipid.columns:
        temp_df = pd.concat([metadata, lipid[[feature]]], axis=1)
        groups = [group[feature].dropna().values for name, group in temp_df.groupby(group_col)]

        # Kruskal-Wallis Test
        stat, p = kruskal(*groups)
        results.append({'Feature': feature, 'Kruskal_Wallis_stat': stat, 'Kruskal_Wallis_pval': p})

    kruskal_df = pd.DataFrame(results)
    kruskal_df['adj_pval'] = multipletests(kruskal_df['Kruskal_Wallis_pval'], method='fdr_bh')[1]

    # Visualize top 3 significant features
    top_features = kruskal_df.sort_values('adj_pval').head(3)['Feature']

    for feat in top_features:
        plt.figure(figsize=(10, 5))
        sns.boxplot(data=df, x=group_col, y=feat, palette='Set2')
        plt.title(f'Lipid Distribution of {feat} by {group_col}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    # Pairwise Wilcoxon (Mann-Whitney U) Test for top features
    pairwise_results = []
    for feature in top_features:
        temp_df = pd.concat([metadata, lipid[[feature]]], axis=1)
        for (g1, g2) in itertools.combinations(temp_df[group_col].unique(), 2):
            group1_vals = temp_df[temp_df[group_col] == g1][feature].dropna()
            group2_vals = temp_df[temp_df[group_col] == g2][feature].dropna()

            if len(group1_vals) > 0 and len(group2_vals) > 0:
                stat, pval = mannwhitneyu(group1_vals, group2_vals, alternative='two-sided')
                pairwise_results.append({
                    'Feature': feature,
                    'Group1': g1,
                    'Group2': g2,
                    'U_stat': stat,
                    'p_value': pval
                })

    pairwise_df = pd.DataFrame(pairwise_results)
    if not pairwise_df.empty:
        pairwise_df['adj_pval'] = multipletests(pairwise_df['p_value'], method='fdr_bh')[1]
    else:
        pairwise_df['adj_pval'] = []

    return kruskal_df.sort_values('adj_pval'), pairwise_df.sort_values('adj_pval')


In [ ]:
kruskal_results_lipid, pairwise_results_lipid = analyze_lipid_by_diabetes_group(lipid_merge)


In [ ]:
print(kruskal_results_lipid.head())
print(pairwise_results_lipid.head())


In [ ]:
lipid_merge

In [ ]:

def correlate_clinical_with_expression(df, clinical_vars=['Age', 'BMI', 'OGTT', 'HbA1c']):
    expression = df.drop(columns=['#NAME', 'Diagnosis'] + clinical_vars)
    clinical = df[clinical_vars]

    corr_results = []

    for clinical_var in clinical.columns:
        for gene in expression.columns:
            corr, pval = spearmanr(df[clinical_var], df[gene])
            corr_results.append({
                'Clinical_Variable': clinical_var,
                'Feature': gene,
                'SpearmanR': corr,
                'p_value': pval
            })

    corr_df = pd.DataFrame(corr_results)
    corr_df['adj_pval'] = multipletests(corr_df['p_value'], method='fdr_bh')[1]
    sig_corrs = corr_df[corr_df['adj_pval'] < 0.05].sort_values('adj_pval')

    # Top heatmap
    top_corrs = sig_corrs.groupby('Clinical_Variable').head(5)

    for var in clinical_vars:
        top_feats = top_corrs[top_corrs['Clinical_Variable'] == var]['Feature']
        if not top_feats.empty:
            data = df[list(top_feats) + [var]]
            corr_matrix = data.corr(method='spearman')
            plt.figure(figsize=(8, 6))
            sns.heatmap(corr_matrix, annot=True, cmap='vlag', center=0)
            plt.title(f'Spearman Correlation: {var} vs Top Features(gene expressions)')
            plt.tight_layout()
            plt.show()

    return sig_corrs
significant_correlations_rnaseq = correlate_clinical_with_expression(rnaseq_merge)
significant_correlations_rnaseq.head(10)


In [ ]:
significant_correlations_lipid = correlate_clinical_with_expression(lipid_merge)
significant_correlations_lipid.head(10)


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# Load lipid + clinical data (make sure lipid_merge.csv exists)
lipid_df = pd.read_csv("lipid_merge.csv")

# Define clinical variables to correlate with
clinical_vars = ['Age', 'BMI', 'OGTT', 'HbA1c']

# Remove non-lipid columns
lipid_features = lipid_df.drop(columns=['#NAME', 'Diagnosis'] + clinical_vars, errors='ignore')

# Compute Spearman correlations for each clinical variable vs each lipid
corr_results = []
for clinical_var in clinical_vars:
    for lipid in lipid_features.columns:
        corr, pval = spearmanr(lipid_df[clinical_var], lipid_df[lipid])
        corr_results.append({
            'Clinical_Variable': clinical_var,
            'Feature': lipid,
            'SpearmanR': corr,
            'p_value': pval
        })

# Create DataFrame and add absolute correlation column
corr_df = pd.DataFrame(corr_results)
corr_df['abs_corr'] = corr_df['SpearmanR'].abs()

# Get top 5 features with highest absolute correlation per clinical variable
top_corrs = corr_df.sort_values('abs_corr', ascending=False).groupby('Clinical_Variable').head(5)

# Generate correlation heatmaps for each clinical variable
for var in clinical_vars:
    top_feats = top_corrs[top_corrs['Clinical_Variable'] == var]['Feature'].tolist()
    if top_feats:
        data = lipid_df[top_feats + [var]]
        corr_matrix = data.corr(method='spearman')
        plt.figure(figsize=(8, 6))
        sns.heatmap(corr_matrix, annot=True, cmap='vlag', center=0)
        plt.title(f'Spearman Correlation: {var} vs Top Lipids')
        plt.tight_layout()
        plt.show()


In [ ]:
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests
import pandas as pd

# Assume: rnaseq_merge has gene features + clinical variables
clinical_vars = ['Age', 'BMI', 'OGTT', 'HbA1c']
expression = rnaseq_merge.drop(columns=['#NAME', 'Diagnosis'] + clinical_vars, errors='ignore')

results_rnaseq = []
for var in clinical_vars:
    for gene in expression.columns:
        corr, pval = spearmanr(rnaseq_merge[var], rnaseq_merge[gene])
        results_rnaseq.append({
            'Clinical_Variable': var,
            'Feature': gene,
            'SpearmanR': corr,
            'p_value': pval
        })

# Create DataFrame and adjust p-values
df_rnaseq_corr = pd.DataFrame(results_rnaseq)
df_rnaseq_corr['adj_pval'] = multipletests(df_rnaseq_corr['p_value'], method='fdr_bh')[1]

# View top N
df_rnaseq_corr.sort_values(by='adj_pval').head(10)


In [ ]:
df_rnaseq_corr.sort_values(by='adj_pval')

In [ ]:
# Assume: lipid_merge has lipid features + clinical variables
lipid_features = lipid_merge.drop(columns=['#NAME', 'Diagnosis'] + clinical_vars, errors='ignore')

results_lipid = []
for var in clinical_vars:
    for lipid in lipid_features.columns:
        corr, pval = spearmanr(lipid_merge[var], lipid_merge[lipid])
        results_lipid.append({
            'Clinical_Variable': var,
            'Feature': lipid,
            'SpearmanR': corr,
            'p_value': pval
        })

# Create DataFrame and adjust p-values
df_lipid_corr = pd.DataFrame(results_lipid)
df_lipid_corr['adj_pval'] = multipletests(df_lipid_corr['p_value'], method='fdr_bh')[1]

# View top N
df_lipid_corr.sort_values(by='adj_pval').head(10)


In [ ]:
def run_pca_clustering_analysis(df, pca_components=2, n_clusters=3):
 
    df.columns = df.columns.astype(str)

    # Drop identifier column if exists
    if '#NAME' in df.columns:
        df = df.drop(columns=['#NAME'])

    # Separate labels
    if 'Diagnosis' not in df.columns:
        raise ValueError("The dataset must contain a 'Diagnosis' column.")

    labels = df['Diagnosis']
    X = df.drop(columns=['Diagnosis'])

    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # PCA
    pca = PCA(n_components=pca_components)
    X_pca = pca.fit_transform(X_scaled)
    explained_var = pca.explained_variance_ratio_

    # KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_ids = kmeans.fit_predict(X_scaled)

    # Map numeric clusters to names
    name_map = {i: f"Cluster {chr(65+i)}" for i in range(n_clusters)}
    cluster_labels_named = pd.Series(cluster_ids).map(name_map)

    # Plot PCA colored by Diagnosis
    plt.figure(figsize=(8,6))
    sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=labels, palette='Set1')
    plt.title("PCA - Colored by Diagnosis")
    plt.xlabel(f"PC1 ({explained_var[0]*100:.1f}% var)")
    plt.ylabel(f"PC2 ({explained_var[1]*100:.1f}% var)")
    plt.grid(True)
    plt.legend(title='Diagnosis', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # Plot PCA colored by Clusters (named)
    plt.figure(figsize=(8,6))
    sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=cluster_labels_named, palette='Set2')
    plt.title(f"KMeans Clusters (k={n_clusters}) on PCA")
    plt.xlabel(f"PC1 ({explained_var[0]*100:.1f}% var)")
    plt.ylabel(f"PC2 ({explained_var[1]*100:.1f}% var)")
    plt.grid(True)
    plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # Optional: compute Adjusted Rand Index
    label_encoded = pd.factorize(labels)[0]
    ari = adjusted_rand_score(label_encoded, cluster_ids)

    return {
        "explained_variance": explained_var,
        "pca_components": X_pca,
        "cluster_ids": cluster_ids,
        "cluster_labels_named": cluster_labels_named.tolist(),
        "adjusted_rand_index": ari
    }


In [ ]:
results_rnaseq = run_pca_clustering_analysis(rnaseq_merge, pca_components=2, n_clusters=3)

print("Explained variance by PC1 & PC2 for rnaseq:", results_rnaseq['explained_variance'])
print("Adjusted Rand Index (label vs clusters) for rnaseq:", results_rnaseq['adjusted_rand_index'])

In [ ]:
results_lipids = run_pca_clustering_analysis(lipid_merge, pca_components=2, n_clusters=3)

print("Explained variance by PC1 & PC2 for lipid:", results_lipids['explained_variance'])
print("Adjusted Rand Index (label vs clusters) for lipid:", results_lipids['adjusted_rand_index'])